# Cleaning the dataset from spotify API containing hit songs (1970-2020)

In [107]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [108]:
df = pd.read_csv('../data/top_hits_1970_to_2019_real.csv')

In [109]:
df.head()

,id,name,artist,popularity,explicit,energy,tempo,positiveness,danceability,acousticness,instrumentalness,loudness,mode,speechiness,duration_ms,key,album_name,year
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,79,False,0.403,143.462,0.410,0.443,0.6310,0.000000,-8.339,1,0.0322,243027,0,Let It Be (Remastered),1970
1,6QhXQOpyYvbpdbyjgAqKdY,Cecilia,Simon & Garfunkel,76,False,0.876,102.762,0.954,0.755,0.3570,0.000005,-8.867,1,0.0362,174827,0,Bridge Over Troubled Water,1970
2,2BhrrVRC8CoiWejbzj0VjQ,Son of a Preacher Man,Aretha Franklin,38,False,0.473,77.583,0.561,0.474,0.5450,0.000001,-11.454,1,0.0601,199267,2,The Queen Of Soul,1970
3,3Bh6uInhcVBVvLraGZdkKD,Who'll Stop The Rain,Creedence Clearwater Revival,65,False,0.797,123.566,0.622,0.598,0.0424,0.000407,-6.793,1,0.0332,147493,7,Cosmo's Factory (Expanded Edition),1970
4,6UkMcAA19lTdjs22jtB7o2,Big Yellow Taxi,Joni Mitchell,2,False,0.470,85.527,0.970,0.611,0.5790,0.000000,-9.135,1,0.0356,134800,4,Ladies of the Canyon,1970


<br>

## Cleaning Summary (TLDR)

- Removed column "instrumentalness" due to high percentage of missing values
- Filtered out songs with a popularity score less than 20
- Renamed column "duration_ms" to "duration_minutes" and converted the values from milliseconds to minutes
- Mapped "key" column values from integers (0-11) to their respective musical keys (0 to C, 1 to Csharp/Dflat, etc)
- Updated "mode" column to represent 'Major' or 'Minor' instead of binary values (1 or 0).

<br>

## 1. Removing columns that contains too many nulls or zeros

### Instrumentalness

I noticed from just looking at the head of the dataframe that the instrumentalness column does not look right, lets see how many 0 values it holds

In [110]:
instrumentalness_zeros = df[df['instrumentalness'] == 0]

In [111]:
instrumentalness_zeros.shape[0]

1844

In [112]:
(1844/5000) * 100

36.88

As there is about 37% values missing from this column, I want to drop it completely

In [113]:
df.drop('instrumentalness', axis=1, inplace=True)

### Speechiness

In [114]:
low_speechiness = df[df['speechiness'] < 0.3]
low_speechiness.shape[0]

4847

Considering that API documentation states that values below 0.33 "represent music and other non-speech-like tracks", I don't think it's accurate that my dataset almost only contains values way below this mark. Let's drop this attribute as well.

In [115]:
df.drop('speechiness', axis=1, inplace=True)

In [116]:
df.describe()

,popularity,energy,tempo,positiveness,danceability,acousticness,loudness,mode,duration_ms,key,year
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03,5000.00000,5000.000000
mean,66.149600,0.664771,119.669626,0.596155,0.631475,0.197982,-7.885891,0.660600,2.442350e+05,5.31220,1994.500000
std,10.759627,0.190074,26.846559,0.240405,0.148229,0.226261,3.450831,0.473553,6.087768e+04,3.59351,14.432313
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-24.709000,0.000000,0.000000e+00,-1.00000,1970.000000
25%,59.000000,0.535000,99.912000,0.408000,0.540000,0.025000,-9.965250,0.000000,2.094098e+05,2.00000,1982.000000
50%,67.000000,0.688000,119.029500,0.615000,0.645000,0.102500,-7.168000,1.000000,2.352200e+05,5.00000,1994.500000
75%,74.000000,0.819000,133.738250,0.798250,0.736000,0.301250,-5.281000,1.000000,2.694000e+05,9.00000,2007.000000
max,99.000000,0.999000,210.857000,0.989000,0.980000,0.978000,0.000000,1.000000,1.561133e+06,11.00000,2019.000000


This should be it for the columns I want to remove

<br>

## 2. Removing rows with zeroes and very low values

When fetching the data I noticed some messages from error handling that there were some rows that did not contain any data. I will select a column like danceability and see if it contains any zeroes.

In [117]:
danceability_zeros = df[df['danceability'] == 0]

In [118]:
danceability_zeros

,id,name,artist,popularity,explicit,energy,tempo,positiveness,danceability,acousticness,loudness,mode,duration_ms,key,album_name,year
2791,5lijIZgEqHF35VSG6WtYCC,You Might Need Somebody,Shola Ama,48,False,0.0,0.0,0.0,0.0,0.0,0.0,0,0,-1,You Might Need Somebody,1997
3453,6MFQeWtk7kxWGydnJB2y36,These Words,Natasha Bedingfield,68,False,0.0,0.0,0.0,0.0,0.0,0.0,0,0,-1,Unwritten,2004


Removing these two rows should not be a problem since it won't really affect the analysis later on.

In [119]:
df = df[df['danceability'] != 0]

I also noticed some songs that had very low popularity, since these playlists are hitlist I want to remove these outliers from the dataset. I think a popularity score like 20 is a good treshold to set.

In [120]:
low_pop = df[df['popularity'] <= 20]

In [121]:
low_pop

,id,name,artist,popularity,explicit,energy,tempo,positiveness,danceability,acousticness,loudness,mode,duration_ms,key,album_name,year
4,6UkMcAA19lTdjs22jtB7o2,Big Yellow Taxi,Joni Mitchell,2,False,0.47,85.527,0.97,0.611,0.579,-9.135,1,134800,4,Ladies of the Canyon,1970


In [122]:
df = df[df['popularity'] > 20]

Lets have a look at the summary statistics again to see if it looks better

In [123]:
df.describe()

,popularity,energy,tempo,positiveness,danceability,acousticness,loudness,mode,duration_ms,key,year
count,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4.997000e+03,4997.000000,4997.000000
mean,66.165699,0.665076,119.724356,0.596319,0.631731,0.197985,-7.888798,0.660796,2.443547e+05,5.314989,1994.502502
std,10.721405,0.189645,26.743248,0.240122,0.147734,0.226230,3.448213,0.473486,6.067971e+04,3.592320,14.431814
min,36.000000,0.026400,50.937000,0.037700,0.135000,0.000005,-24.709000,0.000000,9.286700e+04,0.000000,1970.000000
25%,59.000000,0.535000,99.931000,0.409000,0.540000,0.025000,-9.972000,0.000000,2.094380e+05,2.000000,1982.000000
50%,67.000000,0.688000,119.040000,0.615000,0.645000,0.103000,-7.170000,1.000000,2.352270e+05,5.000000,1994.000000
75%,74.000000,0.819000,133.739000,0.798000,0.736000,0.301000,-5.281000,1.000000,2.694000e+05,9.000000,2007.000000
max,99.000000,0.999000,210.857000,0.989000,0.980000,0.978000,-0.140000,1.000000,1.561133e+06,11.000000,2019.000000


<br>

## 3. Converting the duration of songs (milliseconds to minutes)

Since looking at milliseconds is a bit weird I'd like to convert this value to minutes, something that everyone understands.

In [124]:
df = df.rename(columns={'duration_ms': 'duration_minutes'})

df['duration_minutes'] = df['duration_minutes'] / 60000


Let's also round up these minutes

In [125]:
df['duration_minutes'] = df['duration_minutes'].round(1)

In [126]:
df['duration_minutes'].head(3)

0    4.1
1    2.9
2    3.3
Name: duration_minutes, dtype: float64

<br>

## 4. Changing keys from integers to strings

Next up I would like to change the name of they keys, spotify name these using Pitch Class notation.

<img src="https://davidkulma.com/wp-content/uploads/2016/08/Integer-Circle.001.png" width="300"/>

In [127]:
key_mapping = {
    0: "C",
    1: "Csharp/Dflat",
    2: "D",
    3: "Dsharp/Eflat",
    4: "E",
    5: "F",
    6: "Fsharp/Gflat",
    7: "G",
    8: "Gsharp/Aflat",
    9: "A",
    10: "Asharp/Bflat",
    11: "B"
}

In [128]:
df['key'] = df['key'].map(key_mapping)

In [129]:
df['key'].head()

0               C
1               C
2               D
3               G
5    Gsharp/Aflat
Name: key, dtype: object

That looks better and much easier to follow

In [130]:
df

,id,name,artist,popularity,explicit,energy,tempo,positiveness,danceability,acousticness,loudness,mode,duration_minutes,key,album_name,year
0,7iN1s7xHE4ifF5povM6A48,Let It Be - Remastered 2009,The Beatles,79,False,0.403,143.462,0.410,0.443,0.6310,-8.339,1,4.1,C,Let It Be (Remastered),1970
1,6QhXQOpyYvbpdbyjgAqKdY,Cecilia,Simon & Garfunkel,76,False,0.876,102.762,0.954,0.755,0.3570,-8.867,1,2.9,C,Bridge Over Troubled Water,1970
2,2BhrrVRC8CoiWejbzj0VjQ,Son of a Preacher Man,Aretha Franklin,38,False,0.473,77.583,0.561,0.474,0.5450,-11.454,1,3.3,D,The Queen Of Soul,1970
3,3Bh6uInhcVBVvLraGZdkKD,Who'll Stop The Rain,Creedence Clearwater Revival,65,False,0.797,123.566,0.622,0.598,0.0424,-6.793,1,2.5,G,Cosmo's Factory (Expanded Edition),1970
5,5LxvwujISqiB8vpRYv887S,I Want You Back,The Jackson 5,80,False,0.538,196.605,0.886,0.469,0.3050,-13.559,1,2.9,Gsharp/Aflat,Diana Ross Presents The Jackson 5,1970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2bjUEg4jBtKBlPdNrTAppI,Easier,5 Seconds of Summer,71,False,0.460,176.055,0.623,0.562,0.4760,-4.173,1,2.6,F,CALM,2019
4996,7AODNxFNPnKaz4hURLzw5l,Ladbroke Grove,AJ Tracey,70,False,0.839,133.989,0.727,0.902,0.0933,-9.447,0,3.2,B,AJ Tracey,2019
4997,6QQtHdz4PNJWQX4nrAQGC9,Slide Away,Miley Cyrus,70,False,0.538,148.016,0.289,0.534,0.1730,-6.673,1,3.9,F,SHE IS COMING,2019
4998,6UnCGAEmrbGIOSmGRZQ1M2,Light On,Maggie Rogers,70,False,0.569,102.054,0.399,0.657,0.2010,-6.287,1,3.9,D,Heard It In A Past Life,2019


<br>

## 5. Changing Mode from integers to strings

Currently in the mode column, 1 represents Major and 0 represents Minor

In [131]:
mode_mapping = {
    0: "Minor",
    1: "Major"
}

In [132]:
df['mode'] = df['mode'].map(mode_mapping)

In [133]:
df['mode'].head()

0    Major
1    Major
2    Major
3    Major
5    Major
Name: mode, dtype: object

That's about it, one last thing I want to do is to remove the id column as well, I kept it their if I ever needed to fetch any additional data on the songs but it was not of much help.

<br>

## 6. Dropping the id column

In [134]:
df.drop('id', axis=1, inplace=True)

<br>

## 7. Having a look at the clean dataframe

In [135]:
df

,name,artist,popularity,explicit,energy,tempo,positiveness,danceability,acousticness,loudness,mode,duration_minutes,key,album_name,year
0,Let It Be - Remastered 2009,The Beatles,79,False,0.403,143.462,0.410,0.443,0.6310,-8.339,Major,4.1,C,Let It Be (Remastered),1970
1,Cecilia,Simon & Garfunkel,76,False,0.876,102.762,0.954,0.755,0.3570,-8.867,Major,2.9,C,Bridge Over Troubled Water,1970
2,Son of a Preacher Man,Aretha Franklin,38,False,0.473,77.583,0.561,0.474,0.5450,-11.454,Major,3.3,D,The Queen Of Soul,1970
3,Who'll Stop The Rain,Creedence Clearwater Revival,65,False,0.797,123.566,0.622,0.598,0.0424,-6.793,Major,2.5,G,Cosmo's Factory (Expanded Edition),1970
5,I Want You Back,The Jackson 5,80,False,0.538,196.605,0.886,0.469,0.3050,-13.559,Major,2.9,Gsharp/Aflat,Diana Ross Presents The Jackson 5,1970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Easier,5 Seconds of Summer,71,False,0.460,176.055,0.623,0.562,0.4760,-4.173,Major,2.6,F,CALM,2019
4996,Ladbroke Grove,AJ Tracey,70,False,0.839,133.989,0.727,0.902,0.0933,-9.447,Minor,3.2,B,AJ Tracey,2019
4997,Slide Away,Miley Cyrus,70,False,0.538,148.016,0.289,0.534,0.1730,-6.673,Major,3.9,F,SHE IS COMING,2019
4998,Light On,Maggie Rogers,70,False,0.569,102.054,0.399,0.657,0.2010,-6.287,Major,3.9,D,Heard It In A Past Life,2019


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4997 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              4997 non-null   object 
 1   artist            4997 non-null   object 
 2   popularity        4997 non-null   int64  
 3   explicit          4997 non-null   bool   
 4   energy            4997 non-null   float64
 5   tempo             4997 non-null   float64
 6   positiveness      4997 non-null   float64
 7   danceability      4997 non-null   float64
 8   acousticness      4997 non-null   float64
 9   loudness          4997 non-null   float64
 10  mode              4997 non-null   object 
 11  duration_minutes  4997 non-null   float64
 12  key               4997 non-null   object 
 13  album_name        4997 non-null   object 
 14  year              4997 non-null   int64  
dtypes: bool(1), float64(7), int64(2), object(5)
memory usage: 590.5+ KB


In [137]:
df.describe()

,popularity,energy,tempo,positiveness,danceability,acousticness,loudness,duration_minutes,year
count,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000,4997.000000
mean,66.165699,0.665076,119.724356,0.596319,0.631731,0.197985,-7.888798,4.072403,1994.502502
std,10.721405,0.189645,26.743248,0.240122,0.147734,0.226230,3.448213,1.011492,14.431814
min,36.000000,0.026400,50.937000,0.037700,0.135000,0.000005,-24.709000,1.500000,1970.000000
25%,59.000000,0.535000,99.931000,0.409000,0.540000,0.025000,-9.972000,3.500000,1982.000000
50%,67.000000,0.688000,119.040000,0.615000,0.645000,0.103000,-7.170000,3.900000,1994.000000
75%,74.000000,0.819000,133.739000,0.798000,0.736000,0.301000,-5.281000,4.500000,2007.000000
max,99.000000,0.999000,210.857000,0.989000,0.980000,0.978000,-0.140000,26.000000,2019.000000


## Exporting to csv

In [139]:
df.to_csv('../data/top_hits_1970_to_2019_cleaned.csv', index=False)